In [2]:
import os
import cv2

In [3]:
color = "/Mary_Public/perception_data/open_source_GAN/LLVIP/LLVIP/color"
thermal = "/Mary_Public/perception_data/open_source_GAN/LLVIP/LLVIP/thermal"

def file_counter(path):
    file_count = 0
    for filename in os.listdir(path):
        if os.path.isfile(os.path.join(path, filename)):
            file_count += 1

    print("Number of files:", file_count)

file_counter(color)
file_counter(thermal)

Number of files: 15488
Number of files: 15488


In [4]:
def extractor(file):
    ID = file[-9:-4]
    return ID 
    
def getting_IDs(folder):
    numbers = set() #making this a set not a list 
    for f in os.listdir(folder):
        #print(f)
        ID_set = extractor(f)
        if numbers is not None: 
            numbers.add(ID_set)

    return sorted(numbers, key=float)

ID_color= getting_IDs(color)
ID_thermal= getting_IDs(thermal)
print(ID_color)
print(ID_thermal)

if ID_color == ID_thermal:
    print("all files match, yay")
else:
    print("something wrong")

matching = set(ID_color).intersection(ID_thermal)
print(matching)

['00001', '00002', '00003', '00004', '00005', '00006', '00007', '00008', '00009', '00010', '00011', '00012', '00013', '00014', '00015', '00016', '00017', '00018', '00019', '00020', '00021', '00022', '00023', '00024', '00025', '00026', '00027', '00028', '00029', '00030', '00031', '00032', '00033', '00034', '00035', '00036', '00037', '00038', '00039', '00040', '00041', '00042', '00043', '00044', '00045', '00046', '00047', '00048', '00049', '00050', '00051', '00052', '00053', '00054', '00055', '00056', '00057', '00058', '00059', '00060', '00061', '00062', '00063', '00064', '00065', '00066', '00067', '00068', '00069', '00070', '00071', '00072', '00073', '00074', '00075', '00076', '00077', '00078', '00079', '00080', '00081', '00082', '00083', '00084', '00085', '00086', '00087', '00088', '00089', '00090', '00091', '00092', '00093', '00094', '00095', '00096', '00097', '00098', '00099', '00100', '00101', '00102', '00103', '00104', '00105', '00106', '00107', '00108', '00109', '00110', '00111', 

In [5]:
def minmax(folder):
    min_w, min_h = float(100000), float(100000)
    max_w, max_h = 0,0

    for f in os.listdir(folder):
        path = os.path.join(folder, f)
        img = cv2.imread(path)
        height,width,_ = img.shape
        min_w = min(min_w, width)
        min_h = min(min_h, height)
        max_w = max(max_w, width)
        max_h = max(max_h, height)

    return min_w, min_h, max_w, max_h

color_minandmax = minmax(color)
print(color_minandmax)
thermal_minandmax = minmax(color)
print(thermal_minandmax)

(1280, 1024, 1280, 1024)
(1280, 1024, 1280, 1024)


In [7]:
import os
import cv2


output="/Mary_Public/perception_data/open_source_GAN/LLVIP/LLVIP/spliced"
if not os.path.exists(output):
    os.mkdir(output)
    print("Directory created:", output)
else:
    print("Directory already exists:", output)

folder1_files = sorted([f for f in os.listdir(color) if f.endswith(".jpg")])
folder2_files = sorted([f for f in os.listdir(thermal) if f.endswith(".jpg")])


for i, (file1,file2) in enumerate(zip(folder1_files,folder2_files)):
    image1_path= os.path.join(color, file1)
    image2_path = os.path.join(thermal, file2)

    image1 = cv2.imread(image1_path)
    image1 = cv2.resize(image1, (256, 256)) 
    image2 = cv2.imread(image2_path)
    image2 = cv2.resize(image2, (256, 256)) 


# using hconcat to splice images
    spliced_image = cv2.hconcat([image1, image2])
    output_filename = f"{str(i + 1).zfill(5)}.jpg"
    output_path = os.path.join(output, output_filename)
    cv2.imwrite(output_path, spliced_image)
    print(f"Spliced image saved: {output_path}")

print(f"Processed {len(folder1_files)} image pairs. Output saved to {output}.")


Directory created: /Mary_Public/perception_data/open_source_GAN/LLVIP/LLVIP/spliced
Spliced image saved: /Mary_Public/perception_data/open_source_GAN/LLVIP/LLVIP/spliced/00001.jpg
Spliced image saved: /Mary_Public/perception_data/open_source_GAN/LLVIP/LLVIP/spliced/00002.jpg
Spliced image saved: /Mary_Public/perception_data/open_source_GAN/LLVIP/LLVIP/spliced/00003.jpg
Spliced image saved: /Mary_Public/perception_data/open_source_GAN/LLVIP/LLVIP/spliced/00004.jpg
Spliced image saved: /Mary_Public/perception_data/open_source_GAN/LLVIP/LLVIP/spliced/00005.jpg
Spliced image saved: /Mary_Public/perception_data/open_source_GAN/LLVIP/LLVIP/spliced/00006.jpg
Spliced image saved: /Mary_Public/perception_data/open_source_GAN/LLVIP/LLVIP/spliced/00007.jpg
Spliced image saved: /Mary_Public/perception_data/open_source_GAN/LLVIP/LLVIP/spliced/00008.jpg
Spliced image saved: /Mary_Public/perception_data/open_source_GAN/LLVIP/LLVIP/spliced/00009.jpg
Spliced image saved: /Mary_Public/perception_data/op

In [8]:
# split into train/test/val
import os
import random
from shutil import copy2
from pathlib import Path

input_folder = "/Mary_Public/perception_data/open_source_GAN/LLVIP/LLVIP/spliced"
output_folder = "/Mary_Public/perception_data/open_source_GAN/LLVIP/LLVIP/split"

if not os.path.exists(output_folder):
    os.mkdir(output_folder)
    print("Directory created:", output_folder)
else:
    print("Directory already exists:", output_folder)

train_ratio = 0.85
val_ratio = 0.10
seed = 42  

random.seed(seed)

# here I create output folders
train_folder = os.path.join(output_folder, "train")
val_folder = os.path.join(output_folder, "val")
test_folder = os.path.join(output_folder, "test")
os.makedirs(train_folder, exist_ok=True)
os.makedirs(val_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

all_images = [f for f in os.listdir(input_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))]

# shuffle images
random.shuffle(all_images)

total_images = len(all_images)
train_split = int(total_images * train_ratio)
val_split = int(total_images * (train_ratio + val_ratio))

train_images = all_images[:train_split]
val_images = all_images[train_split:val_split]
test_images = all_images[val_split:]

def copy_images(image_list, destination_folder):
    for image in image_list:
        src_path = os.path.join(input_folder, image)
        dest_path = os.path.join(destination_folder, image)
        copy2(src_path, dest_path)

copy_images(train_images, train_folder)
copy_images(val_images, val_folder)
copy_images(test_images, test_folder)

print(f"Total images: {total_images}")
print(f"Training set: {len(train_images)} images")
print(f"Validation set: {len(val_images)} images")
print(f"Test set: {len(test_images)} images")


Directory created: /Mary_Public/perception_data/open_source_GAN/LLVIP/LLVIP/split
Total images: 15488
Training set: 13164 images
Validation set: 1549 images
Test set: 775 images
